# Transform afids-macaca points between templates via the computed warps

**Authors:** Nikoloz Sirmpilatze (NSirmpilatze@dpz.eu)

**Last updated:** 28 Feb 2020

## Import libraries

In [14]:
import os
import numpy as np
import pandas as pd
from itertools import combinations

## Define paths 

In [17]:
os.chdir(os.getcwd())

# prefixes (RheMAP and AFIDs use different prefixes for the templates)
rhemap_prefixes = ['NMTv1.3', 'D99', 'INIA', 'MNI', 'YRK']
afids_prefixes = ['nmtv1.3', 'd99', 'inia19', 'macaqueMNI', 'yerkes19']
# make dictionary matching RheMAP prefixes to AFIDs prefixes
prefix_dict = {key: value for (key, value) in zip(rhemap_prefixes, afids_prefixes)}

# Warps (Replace with your local RheMAP warp paths)
warps = '/home/nsirmpilatze/home@dpz/RheMAP_data/warps/'
warps_linear = warps + 'linear/' # linear: Affine only
warps_final = warps + 'final/' # final: composite Affine + SyN

# fiducials (stored within the repository)
fid_inputs = './fiducials/PHASE1_PostQC/'
fid_outputs = './fiducials/PHASE1_PostQC_transformed/'

## Define function for transforming fcsv files

In [12]:
def transform_fcsv(input_fcsv, output_fcsv, transform, invert=0):
    '''Applies ANTs transform to 3D slicer fcsv file in 3 steps:
        
    1. Slicer fcsv (RAS) is converted to ANTs-compatible (LPS) csv
    2. The transform is applied to the csv via antsApplyTransformsToPoints
    3. The transformed csv is converted back to Slicer fcsv format
    Function is based on afids-tools legacy scripts github.com/afids/afids-tools

    Parameters
    ----------
    input fcsv: path to input Slicer fcsv file
    ouput fcsv: path to output Slicer fcsv file
    transform: path to ANTs transform file (either linear .mat or .nii.gz warp)
    invert: if 1, linear .mat is inverted
    '''

    # get output directory
    output_dir = os.path.dirname(output_fcsv)
    # temporary csv files are also saved in the same output directory
    orig_csv = os.path.join(output_dir, 'tmp_orig.csv')
    transformed_csv = os.path.join(output_dir, 'tmp_transformed.csv')

    # convert Slicer RAS oriented FCSV (default)
    # to Ants LPS oriented format (expected orientation)
    # use with CAUTION: orientation flips here
    df = pd.read_csv(input_fcsv, skiprows=2)
    coords = df[['x', 'y', 'z']]
    coords.loc[:, 't'] = np.zeros(len(coords)) # add a 4th dimension of zeros 
    coords['x'] = -1 * coords['x'] # flip orientation in x
    coords['y'] = -1 * coords['y'] # flip orientation in y
    coords.to_csv(orig_csv, index=False, float_format='%.3f')

    # apply transforms to original csv and get transformed csv
    !antsApplyTransformsToPoints -d 3 -i $orig_csv -o $transformed_csv -t [$transform, $invert]

    new_coords = pd.read_csv(transformed_csv)
    # flip x and y signs, to convert back from ANTs LPS to slicer RAS space
    df['x'] = -1 * new_coords['x'].values.round(3)
    df['y'] = -1 * new_coords['y'].values.round(3)
    df['z'] = new_coords['z'].values.round(3)

    # read lines from input_fcsv
    with open(input_fcsv, 'r') as file:
        lines = file.readlines()
    # replace data lines (leave header unchanged)
    for i in df.index:
        row_entries = [str(cell) for cell in df.iloc[i, :].values]
        lines[3 + i] = ','.join(row_entries) + '\n'
    # write lines to outout_fcsv
    with open(output_fcsv, 'w') as file:
        file.writelines(lines)

    # remove temporary files
    os.remove(orig_csv)
    os.remove(transformed_csv)

## Apply transforms to points

**Important note**: ANTs transforms on points work in the opposite direction (as compared to image transforms).

For example, let's imagine we have to go from space A to space B either using a linear transform (.mat) or a non-linear warp(.nii.gz)
* Linear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B.mat`
* Linear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B -t [A-to-B.mat, 1]` (inverting the forward transform)
* Nonlinear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B_1Warp.nii.gz` (for RheMAP we would use the `A-to-B_CompositeWarp.nii.gz`)
* Nonlinear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B A-to-B_1InverseWarp.nii.gz` (for RheMAP we would use the `B-to-A_CompositeWarp.nii.gz`)

In [20]:
# Iterate over unique template pairs
for rheA, rheB in combinations(rhemap_prefixes, 2):

    # prefixes
    afdA, afdB = prefix_dict[rheA], prefix_dict[rheB]
    print('Transforming fiducials between {0} and {1}...'.format(afdA, afdB))

    # input fiducials
    fcsv_A = fid_inputs + '{0}_MEAN_QC.fcsv'.format(afdA)
    fcsv_B = fid_inputs + '{0}_MEAN_QC.fcsv'.format(afdB)

    # output fiducials
    fcsv_AinB_linear = fid_outputs + '{0}_MEAN_QC_in_{1}_linear.fcsv'.format(afdA, afdB)
    fcsv_AinB_final = fid_outputs + '{0}_MEAN_QC_in_{1}_final.fcsv'.format(afdA, afdB)
    fcsv_BinA_linear = fid_outputs + '{0}_MEAN_QC_in_{1}_linear.fcsv'.format(afdB, afdA)
    fcsv_BinA_final = fid_outputs + '{0}_MEAN_QC_in_{1}_final.fcsv'.format(afdB, afdA)

    # transforms
    AtoB_linear = warps_linear + '{0}_to_{1}_affine_0GenericAffine.mat'.format(rheA, rheB)
    AtoB_final = warps_final + '{0}_to_others/{0}_to_{1}_CompositeWarp.nii.gz'.format(rheA, rheB)
    BtoA_final = warps_final + '{0}_to_others/{0}_to_{1}_CompositeWarp.nii.gz'.format(rheB, rheA)

    # apply A to B transforms
    transform_fcsv(fcsv_A, fcsv_AinB_linear, AtoB_linear, invert=1)
    print('{0} fiducial linearly transformed to {1} space'.format(afdA, afdB))
    transform_fcsv(fcsv_A, fcsv_AinB_final, BtoA_final)
    print('{0} fiducial nonlinearly transformed to {1} space'.format(afdA, afdB))

    # apply B to A transforms
    transform_fcsv(fcsv_B, fcsv_BinA_linear, AtoB_linear, invert=0)
    print('{0} fiducial linearly transformed to {1} space'.format(afdB, afdA))
    transform_fcsv(fcsv_B, fcsv_BinA_final, AtoB_final)
    print('{0} fiducial nonlinearly transformed to {1} space\n'.format(afdB, afdA))

Transforming fiducials between nmtv1.3 and d99...
nmtv1.3 fiducial linearly transformed to d99 space
nmtv1.3 fiducial nonlinearly transformed to d99 space
d99 fiducial linearly transformed to nmtv1.3 space
d99 fiducial nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and inia19...
nmtv1.3 fiducial linearly transformed to inia19 space
nmtv1.3 fiducial nonlinearly transformed to inia19 space
inia19 fiducial linearly transformed to nmtv1.3 space
inia19 fiducial nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and macaqueMNI...
nmtv1.3 fiducial linearly transformed to macaqueMNI space
nmtv1.3 fiducial nonlinearly transformed to macaqueMNI space
macaqueMNI fiducial linearly transformed to nmtv1.3 space
macaqueMNI fiducial nonlinearly transformed to nmtv1.3 space

Transforming fiducials between nmtv1.3 and yerkes19...
nmtv1.3 fiducial linearly transformed to yerkes19 space
nmtv1.3 fiducial nonlinearly transformed to yerkes19 spa